In [5]:
import numpy as np
from numpy.lib.stride_tricks import as_strided
import torch
import torch.nn as nn
import matplotlib.pyplot as plt 
import scipy.io as sio
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve
from scipy import ndimage, misc
import time
import os    
import random

In [14]:
 
os.environ['KMP_DUPLICATE_LIB_OK']='True'
#%% Check GPU availability
print(f"Pytorch Version: {torch.__version__}")
print("GPU is", "available" if torch.cuda.is_available() else "NOT AVAILABLE")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Pytorch Version: 2.9.0+cu128
GPU is available


In [15]:
#%% set seed

manualSeed = 1234
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(manualSeed)
torch.cuda.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)
np.random.seed(manualSeed)
random.seed(manualSeed)


In [16]:
def pool2d(A, kernel_size, stride, padding=0, pool_mode='max'):
    """
    Perform a 2D pooling operation (max or average) on a 2D array.

    Parameters
    ----------
    A : ndarray
        2D input array to be pooled.
    kernel_size : int
        Size of the pooling window (assumed square).
    stride : int
        Step size between successive pooling windows.
    padding : int, optional
        Number of zero-padding layers added to all sides of `A`. Default is 0.
    pool_mode : {'max', 'avg'}, optional
        Pooling operation to apply:
        - 'max': takes the maximum value in each window.
        - 'avg': takes the average value in each window.

    Returns
    -------
    pooled : ndarray
        The resulting pooled array of shape 
        ((A.shape[0] - kernel_size + 2 * padding) // stride + 1,
         (A.shape[1] - kernel_size + 2 * padding) // stride + 1).
    """   
    # Padding
    A = np.pad(A, padding, mode='constant')

    # Window view of A
    output_shape = ((A.shape[0] - kernel_size) // stride + 1,
                        (A.shape[1] - kernel_size) // stride + 1)
        
    shape_w = (output_shape[0], output_shape[1], kernel_size, kernel_size)
    strides_w = (stride*A.strides[0], stride*A.strides[1], A.strides[0], A.strides[1])
        
    A_w = as_strided(A, shape_w, strides_w)

    # Return the result of pooling
    if pool_mode == 'max':
        return A_w.max(axis=(2, 3))
    elif pool_mode == 'avg':
        return A_w.mean(axis=(2, 3))

In [32]:
def oc(nelx,nely,x,volfrac,dc,dv,g):
	l1=0
	l2=1e9
	move=0.2
	# reshape to perform vector operations
	xnew=np.zeros(nelx*nely)
	while (l2-l1)/(l1+l2)>1e-3:
		lmid=0.5*(l2+l1)
		xnew[:]= np.maximum(0.0,np.maximum(x-move,np.minimum(1.0,np.minimum(x+move,x*np.sqrt(-dc/dv/lmid)))))
		gt=g+np.sum((dv*(xnew-x)))
		if gt>0 :
			l1=lmid
		else:
			l2=lmid
	return (xnew,gt)

In [11]:
#%% Energy-based PINN neural network
class Net(nn.Module):

    def __init__(self, n_input, n_output, n_layer, n_nodes):
        super(Net, self).__init__()
        self.n_layer = n_layer
        self.Input = nn.Linear(n_input, n_nodes)   # linear layer
        nn.init.xavier_uniform_(self.Input.weight) # wigths and bias initiation
        nn.init.normal_(self.Input.bias)
        
        self.Output = nn.Linear(n_nodes, n_output)
        nn.init.xavier_uniform_(self.Output.weight)
        nn.init.normal_(self.Output.bias)

        self.Hidden = nn.ModuleList() # hidden layer list
        for i in range(n_layer):
            self.Hidden.append(nn.Linear(n_nodes, n_nodes))
        for layer in self.Hidden:
            nn.init.xavier_uniform_(layer.weight)
            nn.init.normal_(layer.bias)

    def forward(self, x):
        y = torch.tanh(self.Input(x)) # tanh activation function
        for layer in self.Hidden:
            y = torch.tanh(layer(y))
        y = self.Output(y)
        return y

In [12]:
#%% Automatic differentiation
def derivative(x, Net, func, order):
    
    w = Net(x)*func(x).view(-1,1)
    
    if order == '0':
        return w
    
    else:
        dw_xy = torch.autograd.grad(w, x, torch.ones_like(w), 
                                    retain_graph=True, create_graph=True)
        dw_x = dw_xy[0][:,0].view(-1,1)
        dw_y = dw_xy[0][:,1].view(-1,1)
        
        if order == '1':
            return w, dw_x, dw_y
    
        else:
            dw_xxy = torch.autograd.grad(dw_x, x, torch.ones_like(dw_x), 
                                         retain_graph=True, create_graph=True)
            dw_xx = dw_xxy[0][:,0].view(-1,1)
            dw_xy = dw_xxy[0][:,1].view(-1,1)
            dw_yy = torch.autograd.grad(dw_y, x, torch.ones_like(dw_y), retain_graph=True, 
                                        create_graph=True)[0][:,1].view(-1,1)
            return w, dw_x, dw_y, dw_xx, dw_yy, dw_xy

In [33]:
 
# Domain data
nelx, nely = 40, 20
ndof = 2*(nelx+1)*(nely+1)
x, y = np.meshgrid(np.linspace(0,2,nelx + 1), np.linspace(0,1,nely + 1))
y = np.flipud(y)
data = np.hstack([x.reshape(-1,1, order = 'F'), y.reshape(-1,1, order = 'F')])

# for contourf
X, Y = np.meshgrid(np.linspace(0,2,nelx), np.linspace(0,1,nely))
Y = np.flipud(Y)

# find boundary
idx_f = np.where((data[:,0]==max(data[:,0])) &  (data[:,1]==0.5))
data = torch.tensor(data, dtype=torch.float32, requires_grad=True, device=device)